# SETUP

## imports

In [1]:
import string
import numpy as np
from itertools import cycle
import requests
import collections
from pprint import pprint
import operator
import concurrent
from time import sleep
from MarbleGameList import MarbleGameLinkedList
import re

## constants

In [2]:
lowercase = string.ascii_lowercase
uppercase = string.ascii_uppercase

## helpers

In [3]:
def get_level_input(lvl_num):
    with open('advent_inputs/%d.txt' %lvl_num) as f:
        level_input=f.read()
        return level_input[:-1]
    
def print_result(answer):
    pprint("RESULT: "+str(answer))
    print()
    pprint("TIME"+"."*60)
    
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

# LEVEL 1

## setup

In [5]:
frequencies=get_level_input(1)
frequencies=frequencies.splitlines()
frequencies=[int(x[1:]) if x[0]=='+' else int(x) for x in frequencies]

## part one

In [6]:
%%time
print_result(sum(frequencies))

'RESULT: 500'

'TIME............................................................'
CPU times: user 214 µs, sys: 112 µs, total: 326 µs
Wall time: 273 µs


## part two

In [6]:
%%time
y=set({})
current_sum = 0
for i in cycle(frequencies):
    current_sum+=i
    if current_sum in y:
        print_result(current_sum)
        break
    else:
        y.add(current_sum)

'RESULT: 709'

'TIME............................................................'
CPU times: user 48.2 ms, sys: 5.87 ms, total: 54.1 ms
Wall time: 53.1 ms


# LEVEL 2

## setup

In [7]:
input_boxes=get_level_input(2)
input_boxes=input_boxes.split('\n')
count_boxes = [collections.Counter(a) for a in input_boxes]

## part one

In [8]:
%%time
twos=0
threes=0
for box_id in count_boxes:
    if 2 in box_id.values():
        twos+=1
    if 3 in box_id.values():
        threes+=1
print_result(str(twos*threes))

'RESULT: 6422'

'TIME............................................................'
CPU times: user 555 µs, sys: 97 µs, total: 652 µs
Wall time: 583 µs


## part two

In [9]:
%%time
for i, box_one in enumerate(input_boxes):
    for box_two in input_boxes[i+1:]:
        diff=0
        wrong_letter_index=0
        for j in range(len(box_one)):
            if box_one[j]==box_two[j]: continue
            diff+=1
            wrong_letter_index=j
            if(diff>1): break
        if(diff==1): 
            print_result(box_one[:wrong_letter_index]+box_one[wrong_letter_index+1:])
            raise StopExecution     

'RESULT: qcslyvphgkrmdawljuefotxbh'

'TIME............................................................'


# LEVEL 3

## setup

In [10]:
fabric_list=get_level_input(3).splitlines()
fabric_list=[x.split(" ") for x in fabric_list]

## part one

In [11]:
%%time
fabric=np.zeros((1000,1000))
for i in fabric_list:
    dim=tuple([int(z) for z in i[2][:-1].split(",")])
    size=tuple([int(z) for z in i[3].split("x")])
    fabric[dim[0]:dim[0]+size[0],dim[1]:dim[1]+size[1]]+=1
    NUM_SINGLE_USED=(fabric > 1).sum()
print_result(NUM_SINGLE_USED)

'RESULT: 101565'

'TIME............................................................'
CPU times: user 1.83 s, sys: 11.3 ms, total: 1.85 s
Wall time: 1.86 s


## part two

In [14]:
%%time
for i in fabric_list:
    dim=tuple([int(z) for z in i[2][:-1].split(",")])
    size=tuple([int(z) for z in i[3].split("x")])
    if((fabric[dim[0]:dim[0]+size[0],dim[1]:dim[1]+size[1]]==1).all()):
        print_result(i[0])

'RESULT: #656'

'TIME............................................................'
CPU times: user 19.6 ms, sys: 3.32 ms, total: 23 ms
Wall time: 20.8 ms


# LEVEL 4

## setup

In [15]:
guards = get_level_input(4)
guards = guards.splitlines()
guards.sort()
guards = [[x[0][-2:], x[1][1:]] for x in [j.split(']') for j in guards]]

## part one

In [16]:
%%time

cur_guard=0
guard_to_min={}
for i, log in enumerate(guards):
    if(log[1][0]=='G'):
        cur_guard=int(log[1].split(" ")[1][1:])
        if cur_guard not in guard_to_min.keys():
            guard_to_min[cur_guard]=np.zeros(60)
    elif(log[1][0]=='f'):
        if(int(log[0])<int(guards[i+1][0])):
            guard_to_min[cur_guard][int(log[0]):int(guards[i+1][0])]+=1
        else:
            guard_to_min[cur_guard][int(log[0]):]+=1
            guard_to_min[cur_guard][:int(guards[i+1][0])]+=1
m = max([sum(v) for v in guard_to_min.values()])

for key,value in guard_to_min.items():
    if( m==sum(value)):
        print_result(list(value).index(max(value))*key)

'RESULT: 102688'

'TIME............................................................'
CPU times: user 3.51 ms, sys: 342 µs, total: 3.85 ms
Wall time: 3.56 ms


## part two

In [17]:
%%time

cur_guard=0
guard_to_min={}
for i, log in enumerate(guards):
    if(log[1][0]=='G'):
        cur_guard=int(log[1].split(" ")[1][1:])
        if cur_guard not in guard_to_min.keys():
            guard_to_min[cur_guard]=np.zeros(60)
    elif(log[1][0]=='f'):
        if(int(log[0])<int(guards[i+1][0])):
            guard_to_min[cur_guard][int(log[0]):int(guards[i+1][0])]+=1
        else:
            guard_to_min[cur_guard][int(log[0]):]+=1
            guard_to_min[cur_guard][:int(guards[i+1][0])]+=1
m = max(i for v in guard_to_min.values() for i in v)

for key,value in guard_to_min.items():
    if( m in value ):
        print_result(list(value).index(m)*key)

'RESULT: 56901'

'TIME............................................................'
CPU times: user 3.46 ms, sys: 190 µs, total: 3.65 ms
Wall time: 3.6 ms


# LEVEL 5

## setup

In [18]:
polymers=get_level_input(5)

## part one

In [19]:
%%time
polymer_temp=polymers
i=0
while i!=len(polymer_temp)-1:
    if(abs(ord(polymer_temp[i])-ord(polymer_temp[i+1]))==32):
        polymer_temp=polymer_temp[0:i]+polymer_temp[i+2:]
        i=max(0,i-1)
    else:
        i+=1
print(len(polymer_temp))

9202
CPU times: user 95.2 ms, sys: 5.14 ms, total: 100 ms
Wall time: 99 ms


## part two

In [20]:
%%time
scores=[]
for letter in lowercase:
    polymer_temp=polymers
    i=0
    polymer_temp=polymer_temp.translate(str.maketrans('', '', letter+letter.upper()))
    while i!=len(polymer_temp)-1:
        if(abs(ord(polymer_temp[i])-ord(polymer_temp[i+1]))==32):
            polymer_temp=polymer_temp[0:i]+polymer_temp[i+2:]
            i=max(0,i-1)
        else:
            i+=1
    scores.append(len(polymer_temp))
print_result(min(scores))

'RESULT: 6394'

'TIME............................................................'
CPU times: user 2.21 s, sys: 70.5 ms, total: 2.28 s
Wall time: 2.28 s


# LEVEL 6

## setup

In [6]:
GRID_SIZE=1000
SHIFT_SIZE=50

coords=get_level_input(6)
coords_list=[tuple([int(coord.split(", ")[0]), int(coord.split(", ")[1])]) for coord in coords.splitlines()]
shift_coords_list = [(x[0]+SHIFT_SIZE, x[1]+SHIFT_SIZE) for x in coords_list]

def l1(c1, c2):
    return abs(c1[0]-c2[0])+abs(c1[1]-c2[1])

def get_index_min_l1(pos):
    coord_min=[]
    for coord in shift_coords_list:
        coord_min.append(l1(coord, pos))
    x=np.where(np.array(coord_min) == np.array(coord_min).min())
    if(len(x[0])>1): return 0
    return x[0][0]

def determine_inf_values():
    inf_set=set({})
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if 0 in {i, j} or GRID_SIZE-1 in {i,j}:
                inf_set.add(get_index_min_l1((i,j))+1)
    return inf_set

inf_set=determine_inf_values()

In [129]:
# TEST_COORDS
# GRID_SIZE=20
# SHIFT_SIZE=5
# coords_list=[(1, 1),(1, 6),(8, 3),(3, 4),(5, 5),(8, 9)]
# shift_coords_list = [(x[0]+SHIFT_SIZE, x[1]+SHIFT_SIZE) for x in shift_coords_list]
# coords= set(coord_list)

# print(coords_list)

## part one

### thoughts

- Need to determine which areas are infinite
    - are these automatically the points that have the lowest or highest x or y coordinate?
- Need to limit scope and calculate each grdp points relation to our coordinates
- Assign the minimum distance to a points area score
- Pick the point that has the minimum area score that is not infiinite

### EPIPHANY
CHANGING THE SIZE OF THE GRID WILL NOT CHANGE THE NUMBER OF RESULTS FOR NON-INFINITE OPTIONS. THIS CAN ONLY BE THE CASE FOR NON INF OPTIONS SO TRY TWO GRIDS ONE OF SMALL SIZE, ONF OF LARGER SIZE AND CHOOSE MAX VALUE THAT DOESN"T CHANGE.

**NOTE**: the grid size must expand in all directions for this method to work. This means points need to be recentered

**NOTTE**: easier solutiton, any point that owns one of thte outer rings will be infinite(given a significant enough buffer around all points


In [25]:
%%time

test_grid = np.zeros((GRID_SIZE,GRID_SIZE))
for i, coord in enumerate(shift_coords_list):
    test_grid[coord[0], coord[1]]=i+1
for i in range(GRID_SIZE):
    for j in range(GRID_SIZE):
        if 0 not in {i, j} or GRID_SIZE-1 not in {i,j}:
            min_value=get_index_min_l1((i,j))+1
            if(min_value not in inf_set):
                test_grid[i][j]=min_value
largest_non_inf_area = int(collections.Counter(list(test_grid.flatten())).most_common(2)[1][1])
print_result(largest_non_inf_area)


'RESULT: 3223'

'TIME............................................................'
CPU times: user 37.9 s, sys: 1.02 s, total: 38.9 s
Wall time: 38.3 s


## part two

In [30]:
%%time

THRESHOLD_DISTANCE=10000
SAFE_POINTS=[]
for i in range(GRID_SIZE):
    for j in range(GRID_SIZE):
        if(sum([l1((i,j),x) for x in shift_coords_list])<THRESHOLD_DISTANCE):
            SAFE_POINTS.append((i,j))
print_result(len(SAFE_POINTS))


'RESULT: 40495'

'TIME............................................................'
CPU times: user 21.7 s, sys: 47.5 ms, total: 21.8 s
Wall time: 21.9 s


# LEVEL 7

## setup

In [60]:
steps = get_level_input(7).splitlines()
print(len(steps))
num_steps=26
uppercase = string.ascii_uppercase
locks={}
being_worked_on=set()
_steps=map(to_node, steps)
solution=[]
step_time={key: value for (key, value) in [(ch,(ord(ch)-4)) for ch in uppercase]}
NUM_WORKERS=5
WORK_TIME=[0]*NUM_WORKERS
WORKING_ON=['-']*NUM_WORKERS
seconds_worked=0

for step in _steps:
    locks.setdefault(step[1],[]).append(step[0])

start_nodes = sorted(list(set(list(uppercase))-set(locks.keys())))
next_step = start_nodes
# print(locks)

def to_node(step):
    return(step[1:-1].translate(str.maketrans('','',lowercase)).split())

101


In [13]:
# # temp_test
# steps=[
# "Step C must be finished before step A can begin.",
# "Step C must be finished before step F can begin.",
# "Step A must be finished before step B can begin.",
# "Step A must be finished before step D can begin.",
# "Step B must be finished before step E can begin.",
# "Step D must be finished before step E can begin.",
# "Step F must be finished before step E can begin."
# ]

# num_steps=6

# uppercase="ABCDEF"

# def to_node(step):
#     return(step[1:-1].translate(str.maketrans('','',lowercase)).split())

##  part one

In [62]:
%%time

while(len(solution)<num_steps):
    next_step=list(set(start_nodes)-set(solution))
    for key, dependencies in locks.items():
        if (set(dependencies) <= set(solution)) and key not in solution:
            next_step.append(key)
    solution+=(sorted(next_step)[0])
print_result("".join(solution))

'RESULT: CFGMHNPVAEBWRQDIUSZJYTKLOX'

'TIME............................................................'
CPU times: user 126 µs, sys: 50 µs, total: 176 µs
Wall time: 150 µs


In [61]:
%%time

while(len(solution)<num_steps):
#     print(seconds_worked)
#     print(f"solution => {solution}")
    for key, dependencies in locks.items():
#         print(f"dependency test => {dependencies}:{solution}")
        if (set(dependencies) <= set(solution)) and key not in being_worked_on and key not in next_step:
            next_step.append(key)
    next_step=sorted(next_step)
#     print(f"next step => {next_step}")
    for i, val in enumerate(WORK_TIME):
#         print(f"worker ${i} has {val} time left")
        if WORKING_ON[i]=='-' and next_step:
            letter = next_step[0]
            WORK_TIME[i]+=step_time[letter]
            WORKING_ON[i]=letter
            being_worked_on.add(letter)
            next_step=next_step[1:] 
#         print(WORKING_ON)
        if(WORK_TIME[i]-1==0):
            solution.append(WORKING_ON[i])
            WORKING_ON[i]='-'
    WORK_TIME=[max(0,x-1) for x in WORK_TIME]
    seconds_worked+=1
print(solution)
print_result(seconds_worked)

['C', 'F', 'G', 'M', 'H', 'N', 'P', 'V', 'A', 'E', 'B', 'W', 'R', 'Q', 'D', 'I', 'U', 'S', 'Z', 'J', 'Y', 'T', 'K', 'L', 'O', 'X']
'RESULT: 828'

'TIME............................................................'
CPU times: user 24.2 ms, sys: 988 µs, total: 25.1 ms
Wall time: 24.6 ms


In [35]:
# Ponderings on a threaded solution!

# THREADED SOLUTION

# def work_on_letter(worker, letter):
#     being_worked_on.add(letter)
#     CURRENTLY_FREE[worker]=0
#     print(f"worker {worker} is working on letter {letter}")
#     sleep(step_time[letter])
#     print(f"worker {worker} is done working on letter {letter}")
#     solution.append(letter)
#     CURRENTLY_FREE[worker]=1


# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     while(len(solution)<num_steps):
#         next_step=list(set(next_step)-set(solution))
#         for key, dependencies in locks.items():
#             if (set(dependencies) <= set(solution)) and key not in solution and key not in being_worked_on:
#                 next_step.append(key)
#         next_step=sorted(next_step)
#         for i, val in enumerate(CURRENTLY_FREE):
#             if val==1 and next_step:
#                 executor.submit(work_on_letter, i, next_step[0])
#                 next_step=next_step[1:] 
# print_result("".join(solution))

# LEVEL 8

## setup

In [30]:
tree_test = get_level_input(8).split(" ")
_tree_test = [int(x) for x in tree_test]
print(tree_test[0])
class Node:
    """
    Class Node
    """
    def __init__(self, num_children, num_data):
        self.num_data = num_data
        self.num_children = num_children
        self.meta_data = []
        self.children = []
        
    def __str__(self):
        return f"NUM_CHILDREN: {self.num_children} && NUM_DATA: {self.num_data}"
ROOT = Node(_tree_test[0], _tree_test[1])

9


In [147]:
tree_test ="2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2".split(" ")
_tree_test = [int(x) for x in tree_test]


print(tree_test[0])
class Node:
    """
    Class Node
    """
    def __init__(self, num_children, num_data):
        self.num_data = num_data
        self.num_children = num_children
        self.meta_data = []
        self.children = []
        
    def __str__(self):
        return f"NUM_CHILDREN: {self.num_children} && NUM_DATA: {self.num_data} && MEDATADATA: {self.meta_data}"
ROOT = Node(_tree_test[0], _tree_test[1])

2


## part one

In [35]:
%%time
TOTAL_META =0

def create_tree_find_meta(node):
    global tree_f
    global TOTAL_META
    for i in range(node.num_children):
        num_c=tree_f[0]
        num_m=tree_f[1]
        tree_f=tree_f[2:]
        node.children.append(create_tree(Node(num_c, num_m)))
    node.meta_data, tree_f = tree_f[:node.num_data], tree_f[node.num_data:]
    TOTAL_META+=sum(node.meta_data)
    return node

tree_f = _tree_test[2:]
tree = create_tree_find_meta(ROOT)
print(TOTAL_META)

49180
CPU times: user 92.5 ms, sys: 2.73 ms, total: 95.2 ms
Wall time: 94.5 ms


## part two

In [36]:
%%time

def create_tree(node):
    global tree_f
    for i in range(node.num_children):
        num_c=tree_f[0]
        num_m=tree_f[1]
        tree_f=tree_f[2:]
        node.children.append(create_tree(Node(num_c, num_m)))
    node.meta_data, tree_f = tree_f[:node.num_data], tree_f[node.num_data:]
    return node

TOTAL_META=0
def meta_value(node):
    global TOTAL_META
    if(node.num_children==0):
        TOTAL_META+=sum(node.meta_data)
        return node
    else:
        for value in node.meta_data:
            if(value<=node.num_children):
                meta_value(node.children[value-1])
            
    
tree_f = _tree_test[2:]
tree = create_tree(ROOT)
meta_value(tree)
print_result(TOTAL_META)

'RESULT: 20611'

'TIME............................................................'
CPU times: user 92 ms, sys: 3.25 ms, total: 95.3 ms
Wall time: 94.3 ms


# LEVEL 9

## thoughts and preparation

using a doubly linked list instantly seemed like the cleanest approach because of the rapid insert and removal times near the head. Modified a doubly linked list into a MarbleGameLinkedList class featuring two methods that define the specific insert and removal rules.

Though I am convinced there is a more elegant solution for part two, I am unable to find it.

Also curious as to the comparable speed of a more naive python list implementation. Might check that out in the future. 

## setup

In [13]:
marble_rules = get_level_input(9).split(';')
print(marble_rules)

['473 players', ' last marble is worth 70904 point']


## part one

In [14]:
%%time

game_board = MarbleGameLinkedList()
game_board.add_marble(0)
NUMBER_PLAYERS = int(marble_rules[0].split(" ")[0])
NUMBER_MARBLES = int(marble_rules[1].split(" ")[5])
scores=[0 for x in range(NUMBER_PLAYERS)]

for x in (range(1, NUMBER_MARBLES+1)):
    if(x%23 == 0):
        scores[(x%NUMBER_PLAYERS)]+=x
        scores[(x%NUMBER_PLAYERS)]+=game_board.remove_seven()
    else:
        game_board.add_marble(x)
print(f"max achieved by player #{scores.index(max(scores))+1}")
print_result(max(scores))

max achieved by player #318
'RESULT: 371284'

'TIME............................................................'
CPU times: user 140 ms, sys: 8.34 ms, total: 149 ms
Wall time: 152 ms


## part two

In [15]:
%%time

game_board = MarbleGameLinkedList()
game_board.add_marble(0)
NUMBER_MARBLES = int(marble_rules[1].split(" ")[5])
scores=[0 for x in range(NUMBER_PLAYERS)]
NUMBER_MARBLES = int(marble_rules[1].split(" ")[5])*100

for x in (range(1, NUMBER_MARBLES+1)):
    if(x%23 == 0):
        scores[(x%NUMBER_PLAYERS)]+=x
        scores[(x%NUMBER_PLAYERS)]+=game_board.remove_seven()
    else:
        game_board.add_marble(x)
print(f"max achieved by player #{scores.index(max(scores))+1}")
print_result(max(scores))

max achieved by player #345
'RESULT: 3038972494'

'TIME............................................................'
CPU times: user 16.1 s, sys: 469 ms, total: 16.6 s
Wall time: 16.9 s


# LEVEL 10

### Assumptions
This solitona makes a few assumptions that make it rather inflexible:
- All stars contribute to the final message
- The final size of the message

## setup

In [20]:
stars = get_level_input(10).split('\n')

stars = [re.findall('(\<(.*?)\>)', star) for star in stars]

# turns stars into a list of position speed point pairs. 
stars = [ [list(map(int, star[0][1].split(", "))), tuple(map(int, star[1][1].split(", ")))] for star in stars]

min_x = min(x[0][0] for x in stars)
max_x = max(x[0][0] for x in stars)
min_y = min(y[0][1] for y in stars)
max_y = max(y[0][1] for y in stars)

# make 0 the minimum position for a star (makes it easier to plot)
for star in stars:
    star[0][0]+=abs(min_x)
    star[0][1]+=abs(min_y)
max_x += abs(min_x)
min_x = 0
max_y += abs(min_y)
min_y = 0

REASONABLE_MAX_X = 10
REASONABLE_MAX_Y = 10

print(f"Minimum X: {min_x}")
print(f"Maximum X: {max_x}")
print(f"Minimum Y: {min_y}")
print(f"Maximum Y: {max_y}")

def zero_index_position(stars, max_x, max_y):
    min_x = min(x[0][0] for x in stars)
    max_x = max(x[0][0] for x in stars)
    min_y = min(y[0][1] for y in stars)
    max_y = max(y[0][1] for y in stars)

    # make 0 the minimum position for a star (makes it easier to plot)
    for star in stars:
        star[0][0]-=abs(min_x)
        star[0][1]-=abs(min_y)
    max_x -= abs(min_x)
    min_x = 0
    max_y -= abs(min_y)
    min_y = 0
    return stars, max_x, max_y

def stars_to_set(stars):
    star_set = set((star[0][0], star[0][1]) for star in stars)
    return star_set

def print_grid(stars):
    grid = [['.']*(max_x+1) for i in range(max_y+1)]
    for star in stars_to_set(stars):
        grid[star[1]][star[0]]='X'
    for line in grid:
        print("".join(line))
        
def star_step(stars):
    for star in stars:
        star[0][0]+=star[1][0]
        star[0][1]+=star[1][1]
    return stars


Minimum X: 0
Maximum X: 107161
Minimum Y: 0
Maximum Y: 107109


## part one

In [19]:
%%time
        
while(max_x>REASONABLE_MAX_X and max_y>REASONABLE_MAX_Y):
    stars = star_step(stars)
    stars, max_x, max_y = zero_index_position(stars, max_x, max_y)
print_grid(stars)

XXXXXX..XXXXX.....XX....XXXXX...X....X..X....X.....XXX...XXXX.
.....X..X....X...X..X...X....X..X....X..X....X......X...X....X
.....X..X....X..X....X..X....X...X..X....X..X.......X...X.....
....X...X....X..X....X..X....X...X..X....X..X.......X...X.....
...X....XXXXX...X....X..XXXXX.....XX......XX........X...X.....
..X.....X..X....XXXXXX..X....X....XX......XX........X...X.....
.X......X...X...X....X..X....X...X..X....X..X.......X...X.....
X.......X...X...X....X..X....X...X..X....X..X...X...X...X.....
X.......X....X..X....X..X....X..X....X..X....X..X...X...X....X
XXXXXX..X....X..X....X..XXXXX...X....X..X....X...XXX.....XXXX.
CPU times: user 581 µs, sys: 151 µs, total: 732 µs
Wall time: 610 µs


## part two

In [14]:
%%time
       
SECONDS=0
while(max_x>REASONABLE_MAX_X and max_y>REASONABLE_MAX_Y):
    SECONDS+=1
    stars = star_step(stars)
    stars, max_x, max_y = zero_index_position(stars, max_x, max_y)
print_result(SECONDS)

'RESULT: 0'

'TIME............................................................'
CPU times: user 477 µs, sys: 294 µs, total: 771 µs
Wall time: 556 µs


# LEVEL 11

## setup

In [18]:
polymers=get_level_input(5)

## part one

In [19]:
%%time
polymer_temp=polymers
i=0
while i!=len(polymer_temp)-1:
    if(abs(ord(polymer_temp[i])-ord(polymer_temp[i+1]))==32):
        polymer_temp=polymer_temp[0:i]+polymer_temp[i+2:]
        i=max(0,i-1)
    else:
        i+=1
print(len(polymer_temp))

9202
CPU times: user 95.2 ms, sys: 5.14 ms, total: 100 ms
Wall time: 99 ms


## part two

In [20]:
%%time
scores=[]
for letter in lowercase:
    polymer_temp=polymers
    i=0
    polymer_temp=polymer_temp.translate(str.maketrans('', '', letter+letter.upper()))
    while i!=len(polymer_temp)-1:
        if(abs(ord(polymer_temp[i])-ord(polymer_temp[i+1]))==32):
            polymer_temp=polymer_temp[0:i]+polymer_temp[i+2:]
            i=max(0,i-1)
        else:
            i+=1
    scores.append(len(polymer_temp))
print_result(min(scores))

'RESULT: 6394'

'TIME............................................................'
CPU times: user 2.21 s, sys: 70.5 ms, total: 2.28 s
Wall time: 2.28 s
